In [45]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)

key = os.getenv('OPENAI_API_KEY')
tavily_key = os.getenv('TAVILY_API_KEY')

In [46]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

In [47]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

In [48]:
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_teddynote.tools.tavily import TavilySearch

In [49]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
import json

In [50]:
##### 1. 상태 정의 #####
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [51]:
##### 2. 도구 정의 및 바인딩 #####
tool = TavilySearch(max_results=1)      # 검색 도구
tools = [tool]                          # 도구 목록에 넣기

In [52]:
##### 3. LLM을 도구와 결합 #####
llm = ChatOpenAI(
    api_key=key, 
    model='gpt-4o-mini'
)

llm_with_tools = llm.bind_tools(tools)  # 도구를 사용할 수 있는 llm

In [53]:
def chatbot(state: State):
    answer = llm_with_tools.invoke(state['messages'])
    return {'messages': [answer]}

In [56]:
# 테스트
quesiton = HumanMessage(content='대한민국 수도는 어디야?')
state1 = State(messages=[quesiton])
print(state1)

answer = chatbot(state1)
print(answer)

{'messages': [HumanMessage(content='대한민국 수도는 어디야?', additional_kwargs={}, response_metadata={})]}
{'messages': [AIMessage(content='대한민국의 수도는 서울입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 98, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgpCj7yD8qjoVar9bUloopw34AgeJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b2ad430e-9258-4a53-b735-6afaa2c1dda9-0', usage_metadata={'input_tokens': 98, 'output_tokens': 9, 'total_tokens': 107, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}
